In [1]:
import json
from pymot.pymot import MOTEvaluation

total = 0 

# Load ground truth data from a JSON file
with open('gt.json', 'r') as f:
    print('reading GT file')
    groundtruth = json.load(f)
    groundtruth=groundtruth[0]
    '''
    for frame in groundtruth['frames']:
        new_groundtruths = frame["annotations"]
        total += len(new_groundtruths)
        #print("Annotations for frame:", new_groundtruths)
    print("Total annotations:", total)
    '''


# Load hypotheses data from a JSON file
with open('yolo.json', 'r') as f:
    hypotheses = json.load(f)
    hypotheses=hypotheses[0]
    # print("hypotheses", hypotheses)
    
overlap_threshold = 0.2
# print(groundtruth)
# Create an instance of the MOTEvaluation class with the loaded data
evaluator = MOTEvaluation(groundtruth, hypotheses,overlap_threshold)
evaluator.evaluate()
# Calculate metrics


mota = evaluator.getMOTA()
motp = evaluator.getMOTP()
absolute_statistics = evaluator.getAbsoluteStatistics()
relative_statistics = evaluator.getRelativeStatistics()

found 2 > 1 ground truth tracks for id vehicle
found 2 > 1 ground truth tracks for id vehicle
found 2 > 1 ground truth tracks for id vehicle


reading GT file
in evaluate
length of groundtruth in evaluate frame 2
hypotheses_frames 1
get hypo {'timestamp': 259.86787336692214, 'num': 1648, 'class': 'frame', 'hypotheses': [{'height': 48, 'width': 51, 'id': 'vehicle', 'y': 362, 'x': 423, 'type': 'hypothesis', 'class': 'unevaluated'}]}
  munkres_matrix [[9223372036854775807], [9223372036854775807]]
GT in for loop - corres {'dco': True, 'height': 43, 'width': 52, 'id': 'vehicle', 'y': 360, 'x': 422, 'type': 'groundtruth', 'class': 'unevaluated'}
hypo hungariam mat {'height': 48, 'width': 51, 'id': 'vehicle', 'y': 362, 'x': 423, 'type': 'hypothesis', 'class': 'unevaluated'}
------------
rect_groundtruth (id, x,y,w,h) = (vehicle, 422.0, 360.0, 52.0, 43.0, DCO)
rect_hypothesis (id, x,y,w,h) = (vehicle, 423.0, 362.0, 51.0, 48.0, non-DCO)
overlap 0.8064018511376784
------------
mat  1.2400765184122429
GT in for loop - corres {'dco': True, 'height': 12, 'width': 14, 'id': 'vehicle', 'y': 384, 'x': 506, 'type': 'groundtruth', 'class': 'un

AssertionError: 